In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta




test = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\COVID_QA_Final Output_Testing_details.xlsx')


ignore_state_list = ['SC']

n2w_dict = {0: 'none',1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
             6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}

num2words = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
             6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', \
            19: 'Nineteen', 20: 'Twenty', 30: 'Thirty', 40: 'Forty', \
            50: 'Fifty', 60: 'Sixty', 70: 'Seventy', 80: 'Eighty', \
            90: 'Ninety', 0: 'Zero'}

def n2w(n):
  try:
    return (num2words[n])
  except KeyError:
    try:
      return num2words[n-n%10] + num2words[n%10].lower()
    except KeyError:
      return 'Number out of range'
## Syed-8/20/2020: code to convert numbers below hundred to words
        
def us_states_positivity_greater(df,threshold,ignore_state_list,ignore_states_flag):
    if ignore_states_flag == 1:
      df = df[~df['State'].isin(ignore_state_list)]
    else:
      print('No states removed!')
    greater_than_10pc = df[(df['Join Constant'] == 97) & (df['7 day positivity'] >= threshold)]['State'].tolist()
    greater_than_10pc_values = df[(df['Join Constant'] == 97) & (df['7 day positivity'] >= threshold)]['7 day positivity'].tolist()
    greater_than_10pc_prev_mon_values =  df[(df['Join Constant'] == 98) & (df['State'].isin(greater_than_10pc))]['7 day positivity'].tolist()
    
    pos_greater_pc_dict = {}
    for i in range(0,len(greater_than_10pc)):
        pos_greater_pc_dict[greater_than_10pc[i]] = [greater_than_10pc_values[i],greater_than_10pc_prev_mon_values[i]]
        pos_greater_pc_dict = {k: v for k, v in sorted(pos_greater_pc_dict.items(),reverse = True, key=lambda x: x[1])}
    
    number_states = len(pos_greater_pc_dict)

    expression = []
    for i in range(0, len(pos_greater_pc_dict),1):
        expression.append(list(pos_greater_pc_dict)[i])
        expression.append(float(list(pos_greater_pc_dict.values())[i][0]))
        expression.append(list(pos_greater_pc_dict.values())[i][1])
        #print(expression)
    for i in range(0, len(pos_greater_pc_dict),1):
        expression.append(list(pos_greater_pc_dict.values())[i][1])
   
    if len(pos_greater_pc_dict) == 0:
        for i in range(0,10):
            pos_greater_pc_dict[int(i)] = int(i)
        a = ''
            
    elif len(pos_greater_pc_dict) == 1:
        a = '{} ({:.1f}% vs. {:.1f}% previous month) had a 7-day average positivity rate greater than ' + str(threshold) + '%.'.format(*expression)
    elif len(pos_greater_pc_dict) == 2:
        a = '{} ({:.1f}% vs. {:.1f}% previous month) and {} ({:.1f}% vs. {:.1f}%) had 7-day average positivity rates greater than ' + str(threshold) + '%.'.format(*expression)
    
    elif len(pos_greater_pc_dict) >2:
        comment_length = len(pos_greater_pc_dict) - 2
        #comment  = '{} ({:.1f}% vs. {:.1f}% previous month)' + comment_length*(', {} ({:.1f}% vs. {:.1f}%)') + ' and {} ({:.1f}% vs. {:.1f}%) had 7-day average positivity rates greater than ' + str(threshold) + '%.'.format(*expression)
       
        ## Syed-8/20/2020: code to format comments as per request on 8/19
        #comment=n2w(len(pos_greater_pc_dict))+" States had a 7-day average positivity rate greater than 12%: " + comment_length*('{} ({:.1f}% vs. {:.1f}%)') + ' and {} ({:.1f}% vs. {:.1f}%).'.format(*expression)
        comment=n2w(len(pos_greater_pc_dict))+" States had a 7-day average positivity rate greater than 20%: " + '{} ({:.1f}% vs. {:.1f}% previous month)' + comment_length*(', {} ({:.1f}% vs. {:.1f}%)') + ' and {} ({:.1f}% vs. {:.1f}%)' + str("") + '.'.format(*expression)
     
        a = comment.format(*expression)
    
    return a

States_Comments_Positivity = us_states_positivity_greater(test, 20,ignore_state_list,0)
print(States_Comments_Positivity)

No states removed!
Seven States had a 7-day average positivity rate greater than 20%: MS (100.0% vs. 4.4% previous month), SD (39.9% vs. 24.2%), ID (33.5% vs. 19.8%), WY (29.0% vs. 12.2%), IA (25.0% vs. 15.8%), AL (24.0% vs. 13.3%) and NE (20.8% vs. 13.6%).
